In [2]:
import pandas as pd

def load_data(master_path, code_a_path, code_b_path):
    master = pd.read_csv(master_path)
    code_a = pd.read_csv(code_a_path)
    code_b = pd.read_csv(code_b_path)
    return master, code_a, code_b

def build_code_dicts(code_b):
    code_dicts = {}
    for code in code_b['cd_a'].unique():
        sub = code_b[code_b['cd_a'] == code][['cd_b', 'cd_nm']].copy()
        sub['cd_b'] = pd.to_numeric(sub['cd_b'], errors='coerce').astype('Int64')
        code_dicts[code] = sub.set_index('cd_b')['cd_nm'].to_dict()
    return code_dicts

def map_codes(df, mapping_plan, code_dicts):
    for col, code_key in mapping_plan.items():
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        df[col] = df[col].map(code_dicts.get(code_key, {}))
    return df

def rename_columns(df, rename_map):
    return df.rename(columns=rename_map)

def preprocess_traveller_data(master_path, code_a_path, code_b_path, output_path=None):
    # 1. Load
    df, code_a, code_b = load_data(master_path, code_a_path, code_b_path)
    
    # 2. Build code dictionaries
    code_dicts = build_code_dicts(code_b)

    # 3. Define mapping plan (col: code_key)
    mapping_plan = {
        'HOUSE_INCOME': 'INC',
        'INCOME': 'INC',
        'EDU_NM': 'EDU',
        'EDU_FNSH_SE': 'EFS',
        'MARR_STTS': 'MAR',
        'JOB_NM': 'JOB',
        'JOB_ETC': 'JOE',
        'TRAVEL_TERM': 'TTM',
        'TRAVEL_STYL_1': 'TSY', 'TRAVEL_STYL_2': 'TSY', 'TRAVEL_STYL_3': 'TSY',
        'TRAVEL_STYL_4': 'TSY', 'TRAVEL_STYL_5': 'TSY', 'TRAVEL_STYL_6': 'TSY',
        'TRAVEL_STYL_7': 'TSY', 'TRAVEL_STYL_8': 'TSY',
        'TRAVEL_MOTIVE_1': 'TMT', 'TRAVEL_MOTIVE_2': 'TMT', 'TRAVEL_MOTIVE_3': 'TMT'
    }

    # 4. Map codes
    df = map_codes(df, mapping_plan, code_dicts)

    # 5. Rename columns
    rename_map = {
        "TRAVELER_ID": "여행객ID",
        "RESIDENCE_SGG_CD": "거주시군구코드",
        "GENDER": "성별",
        "AGE_GRP": "연령대",
        "EDU_NM": "최종학력명",
        "EDU_FNSH_SE": "최종학력여부",
        "MARR_STTS": "결혼상태",
        "FAMILY_MEMB": "가구원수",
        "JOB_NM": "직업",
        "JOB_ETC": "직업_기타",
        "INCOME": "본인소득",
        "HOUSE_INCOME": "가구소득",
        "TRAVEL_TERM": "여행일수_기간",
        "TRAVEL_NUM": "여행빈도",
        "TRAVEL_LIKE_SIDO_1": "선호여행_시도_1",
        "TRAVEL_LIKE_SGG_1": "선호여행_시군구_1",
        "TRAVEL_LIKE_SIDO_2": "선호여행_시도_2",
        "TRAVEL_LIKE_SGG_2": "선호여행_시군구_2",
        "TRAVEL_LIKE_SIDO_3": "선호여행_시도_3",
        "TRAVEL_LIKE_SGG_3": "선호여행_시군구_3",
        "TRAVEL_STYL_1": "여행스타일1", "TRAVEL_STYL_2": "여행스타일2",
        "TRAVEL_STYL_3": "여행스타일3", "TRAVEL_STYL_4": "여행스타일4",
        "TRAVEL_STYL_5": "여행스타일5", "TRAVEL_STYL_6": "여행스타일6",
        "TRAVEL_STYL_7": "여행스타일7", "TRAVEL_STYL_8": "여행스타일8",
        "TRAVEL_STATUS_RESIDENCE": "여행출발_거주지",
        "TRAVEL_STATUS_DESTINATION": "여행출발_도착지",
        "TRAVEL_STATUS_ACCOMPANY": "여행동행_유형",
        "TRAVEL_STATUS_YMD": "여행출발일자",
        "TRAVEL_MOTIVE_1": "여행동기1", "TRAVEL_MOTIVE_2": "여행동기2", "TRAVEL_MOTIVE_3": "여행동기3",
        "TRAVEL_COMPANIONS_NUM": "여행동반자수"
    }
    df = rename_columns(df, rename_map)

    # 6. Save if output_path is given
    if output_path:
        df.to_csv(output_path, index=False, encoding='utf-8')
    
    return df


In [5]:
preprocess_traveller_data("C:/Users/User/Desktop/miniproject/where2go/data/tn_traveller_master_여행객 Master_B.csv",
                          "C:/Users/User/Desktop/miniproject/where2go/data/tc_codea_코드A.csv",
                          "C:/Users/User/Desktop/miniproject/where2go/data/tc_codeb_코드B.csv",
                          "C:/Users/User/Desktop/miniproject/where2go/data/traveller_preprocessed_East.csv"
            )

,여행객ID,거주시군구코드,성별,연령대,최종학력명,최종학력여부,결혼상태,가구원수,직업,직업_기타,...,여행스타일7,여행스타일8,여행출발_거주지,여행출발_도착지,여행동행_유형,여행출발일자,여행동기1,여행동기2,여행동기3,여행동반자수
0,b009514,31,여,20,대학교(4년제 이상),졸업,미혼,3,사무 종사자,NaN,...,자연 선호 중간선호,중립,울산광역시,경남,2인 여행(가족 외),2022-10-30~2022-10-30,"운동, 건강 증진 및 충전","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",NaN,1
1,b015795,29,여,30,대학교(4년제 이상),졸업,미혼,1,사무 종사자,NaN,...,도시 선호 중간선호,도시 선호 중간선호,광주광역시,대구,나홀로 여행,2022-10-29~2022-10-29,"역사 탐방, 문화적 경험 등 교육적 동기","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",0
2,b004208,31,여,20,대학교(4년제 이상),졸업,미혼,4,전문가 및 관련 종사자,NaN,...,도시 선호 중간선호,도시 선호 약간선호,울산광역시,경북,2인 여행(가족 외),2022-09-24~2022-09-25,여행 동반자와의 친밀감 및 유대감 증진,새로운 경험 추구,SNS 사진 등록 등 과시,1
3,b006840,11,여,30,대학교(4년제 이상),졸업,미혼,4,사무 종사자,NaN,...,자연 선호 매우선호,도시 선호 매우선호,서울특별시,강원,나홀로 여행,2022-10-05~2022-10-05,SNS 사진 등록 등 과시,새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1
4,b002386,26,남,40,고등학교,졸업,기혼,4,판매 종사자,NaN,...,도시 선호 매우선호,도시 선호 매우선호,부산광역시,울산,자녀 동반 여행,2022-08-22~2022-08-23,여행 동반자와의 친밀감 및 유대감 증진,새로운 경험 추구,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,b000411,11,남,20,대학교(4년제 이상),재학,미혼,3,NaN,학생,...,중립,중립,서울특별시,강원,2인 여행(가족 외),2022-08-26~2022-08-28,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,1
3196,b004476,31,남,20,대학교(4년제 이상),졸업,미혼,2,전문가 및 관련 종사자,NaN,...,도시 선호 중간선호,도시 선호 약간선호,울산광역시,부산,2인 여행(가족 외),2022-09-22~2022-09-23,"역사 탐방, 문화적 경험 등 교육적 동기",새로운 경험 추구,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1
3197,b015777,11,남,50,대학교(4년제 이상),졸업,기혼,4,전문가 및 관련 종사자,NaN,...,자연 선호 중간선호,도시 선호 매우선호,서울특별시,강원,2인 가족 여행,2022-10-30~2022-10-30,새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,"운동, 건강 증진 및 충전",1
3198,b007728,11,남,40,대학교(4년제 이상),중퇴,기혼,3,단순노무종사자,NaN,...,자연 선호 약간선호,자연 선호 약간선호,서울특별시,경북,3인 이상 여행(가족 외),2022-10-21~2022-10-21,여행 동반자와의 친밀감 및 유대감 증진,"역사 탐방, 문화적 경험 등 교육적 동기","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",2
